In [1]:
import pandas as pd
import re

In [2]:
csv_files = [
    'Scraped data/company_data/musicconnection/extracted_data/Apr23_Directory_Recording_Studios.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Aug23_Gear_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Aug23_Rehearsal_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Feb24_ProducerDirectory.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Jan24_GuideAttorneys.csv',
    'Scraped data/company_data/musicconnection/extracted_data/July23_Managers_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Jun23_MuEd_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Jun23_Vocal_Coach_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Mar24_Indie_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Nov23_Publishing_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Oct23_Mastering_Guide.csv',
    'Scraped data/company_data/musicconnection/extracted_data/Sept23_FilmTV_Guide.csv'
]

In [5]:
def process_csv(file_path):
    df = pd.read_csv(file_path)

    # Your function to extract phone numbers
    def extract_phone_number(s):
        if isinstance(s, str):
            phone_regex = r'\b(\d{1,3}\D*){10}\b'
            phone_numbers = re.findall(phone_regex, s)
            # Remove non-digit characters from the phone numbers
            phone_numbers = [''.join(filter(str.isdigit, phone)) for phone in phone_numbers]
            return phone_numbers if phone_numbers else None
        else:
            return None


    # Apply the function to the 'Address/Phone' column
    df['Phone'] = df['Address/Phone'].apply(extract_phone_number)

    # Subtract the 'Phone' column from the 'Address/Phone' column to get the 'Address' column
    def remove_phone_numbers(row):
        if row['Phone'] is None:
            return row['Address/Phone']
        else:
            address = row['Address/Phone']
            for phone_number in row['Phone']:
                address = address.replace(phone_number, '')
            return address.strip()  # Remove leading/trailing whitespace

    def remove_unwanted_substrings(s):
        if isinstance(s, str):
            s = s.replace(' x ', '')
            s = re.sub(r'(?i)fax', '', s)  # Remove 'fax' (case insensitive)
        return s


    df['Address'] = df.apply(remove_phone_numbers, axis=1)
    df['Address'] = df['Address'].apply(remove_unwanted_substrings)

    # Get the position of the old 'Address/Phone' column
    idx = df.columns.get_loc('Address/Phone')

    # Insert the 'Address' and 'Phone' columns at the right position
    df.insert(idx, 'Address_new', df['Address'])
    df.insert(idx + 1, 'Phone_new', df['Phone'])

    # Delete the old 'Address/Phone', 'Address', and 'Phone' columns
    df = df.drop(['Address/Phone', 'Address', 'Phone'], axis=1)

    # Rename the new 'Address' and 'Phone' columns
    df = df.rename(columns={'Address_new': 'Address', 'Phone_new': 'Phone'})

    # Overwrite the original CSV file
    df.to_csv('test.csv', index=False)

# Process all CSV files
#for file_path in csv_files:
process_csv(csv_files[0])



AttributeError: 'float' object has no attribute 'replace'